In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

models = ['model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

        # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}

        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    training_texts.append(preprocess_text(json_to_text(file.read())))
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                nertagger.tag(text)
                text.add_layer(flatten(text['ner'], 'flat_ner'))

                for x in removed_layers:
                    text.pop_layer(x)

                text_to_json(text, file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 151.11203360557556 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 22668
Seconds required: 0.253

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 3573.642877
Trial #2 (eta = 0.200000): 5426.293148
Trial #3 (eta = 0.400000): 10883.912377
Trial #4 (eta = 0.800000): 17639.513200
Trial #5 (eta = 1.600000): 36000.812280 (worse)
Trial #6 (eta = 0.050000): 3143.291648
Tria

***** Epoch #35 *****
Loss: 4368.248482
Improvement ratio: 0.129563
Feature L2-norm: 124.595553
Learning rate (eta): 0.049826
Total number of feature updates: 568505
Seconds required for this iteration: 0.238

***** Epoch #36 *****
Loss: 4197.307459
Improvement ratio: 0.279456
Feature L2-norm: 125.906896
Learning rate (eta): 0.049821
Total number of feature updates: 584748
Seconds required for this iteration: 0.256

***** Epoch #37 *****
Loss: 4120.613276
Improvement ratio: 0.167725
Feature L2-norm: 127.199670
Learning rate (eta): 0.049816
Total number of feature updates: 600991
Seconds required for this iteration: 0.253

***** Epoch #38 *****
Loss: 4029.372252
Improvement ratio: 0.222794
Feature L2-norm: 128.449392
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.236

***** Epoch #39 *****
Loss: 4002.465674
Improvement ratio: 0.163333
Feature L2-norm: 129.698855
Learning rate (eta): 0.049806
Total number of feature updates: 6

Märgendatud fail Tartu_Kodavere_Alatskivi_id9028_1865a.json
Märgendatud fail Harju_Juuru_Kaiu_id1203_1895a.json
Märgendatud fail V6ru_R6uge_Saaluse_id10045_1879a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17204_1884a.json
Märgendatud fail Tartu_Torma_Avinurme_id20727_1871a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24539_1888a.json
Märgendatud fail Tartu_R6ngu_Aakre_id14645_1829a.json
Märgendatud fail Tartu_V6nnu_Ahja_id19396_1887a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id6107_1864a.json
Märgendatud fail Tartu_Kodavere_Ranna_id14144_1855a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17503_1888a.json
Märgendatud fail Harju_Hageri_Kohila_id4769_1886a.json
Märgendatud fail J2rva_Tyri_Kirna_id24911_1881a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20077_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id9677_1871a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id1315_1884a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1603_1878a.json
Märgendatud fail J2rva_Tyri_S2revere_id12869_1879a.json
Märg

Märgendatud fail Tartu_V6nnu_Ahja_id17321_1885a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id4876_1863a.json
Märgendatud fail Harju_Juuru_Kaiu_id18571_1873a.json
Märgendatud fail V6ru_P6lva_Kiuma_id1633_1880a.json
Märgendatud fail V6ru_Vastseliina_Misso_id25270_1886a.json
Märgendatud fail V6ru_Vastseliina_Misso_id10250_1886a.json
Märgendatud fail Viljandi_Suure-Jaani_Syrgavere_id20998_1877a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16934_1886a.json
Märgendatud fail J2rva_Ambla_Uudekyla_id13485_1867a.json
Märgendatud fail V6ru_P6lva_Peri_id10138_1891a.json
Märgendatud fail Viljandi_Kolga-Jaani_Paenasti_id330_1874a.json
Märgendatud fail Tartu_Torma_Avinurme_id17385_1871a.json
Märgendatud fail V6ru_Vastseliina_Misso_id18830_1882a.json
Märgendatud fail Tartu_Torma_Avinurme_id4086_1858a.json
Märgendatud fail J2rva_Tyri_S2revere_id10947_1871a.json
Märgendatud fail Harju_Kose_Habaja_id735_1874a.json
Märgendatud fail Viru_Haljala_Vihula_id10570_1867a.json
Märgendatud fail Harju_Kose_Triigi_i

***** Epoch #20 *****
Loss: 5550.510005
Improvement ratio: 0.330641
Feature L2-norm: 99.355119
Learning rate (eta): 0.049900
Total number of feature updates: 324020
Seconds required for this iteration: 0.240

***** Epoch #21 *****
Loss: 5379.871362
Improvement ratio: 0.304294
Feature L2-norm: 101.322605
Learning rate (eta): 0.049895
Total number of feature updates: 340221
Seconds required for this iteration: 0.230

***** Epoch #22 *****
Loss: 5222.590885
Improvement ratio: 0.289319
Feature L2-norm: 103.199471
Learning rate (eta): 0.049890
Total number of feature updates: 356422
Seconds required for this iteration: 0.239

***** Epoch #23 *****
Loss: 5383.175638
Improvement ratio: 0.227930
Feature L2-norm: 105.111413
Learning rate (eta): 0.049885
Total number of feature updates: 372623
Seconds required for this iteration: 0.239

***** Epoch #24 *****
Loss: 4905.670821
Improvement ratio: 0.303155
Feature L2-norm: 106.835178
Learning rate (eta): 0.049880
Total number of feature updates: 38

...

Märgendatud fail Harju_Juuru_Juuru_id19472_1887a.json
Märgendatud fail Harju_Kose_Palvere_id18184_1882a.json
Märgendatud fail Tartu_Laiuse_Kivij2rve_id4917_1863a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id21217_1875a.json
Märgendatud fail J2rva_Tyri_S2revere_id14565_1886a.json
Märgendatud fail V6ru_R2pina_R2pina_id10635_1868a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id14511_1876a.json
Märgendatud fail Tartu_Kodavere_Pala_id22058_1871a.json
Märgendatud fail V6ru_Kanepi_Krootuse_id25390_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14012_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id22167_1881a.json
Märgendatud fail Tartu_Kodavere_Pala_id22543_1871a.json
Märgendatud fail Viljandi_Paistu_Holstre_id9042_1836a.json
Märgendatud fail Tartu_Kodavere_Pala_id20291_1867a.json
Märgendatud fail Viljandi_P6ltsamaa_Adavere_id20855_1896a.json
Märgendatud fail V6ru_R2pina_Kahkva_id5748_1887a.json
Märgendatud fail Harju_Hageri_Kohila_id10684_1872a.json
Märgendatud fail Tartu_N6o_P

Märgendatud fail Viljandi_P6ltsamaa_Uue-P6ltsamaa_id8714_1854a.json
Märgendatud fail V6ru_P6lva_Kiuma_id9794_1881a.json
Märgendatud fail Tartu_V6nnu_Kiidj2rve_id24401_1865a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16408_1885a.json
Märgendatud fail J2rva_Tyri_Tyri-Alliku_id3994_1903a.json
Märgendatud fail P2rnu_Audru_V6lla_id5150_1877a.json
Märgendatud fail V6ru_R2pina_Kahkva_id8829_1888a.json
Märgendatud fail L22ne_Karuse_Saastna_id22146_1867a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22857_1886a.json
Märgendatud fail Tartu_V6nnu_Ahja_id12159_1873a.json
Märgendatud fail Tartu_Torma_Avinurme_id5061_1860a.json
Märgendatud fail Tartu_R6ngu_Aakre_id6659_1826a.json
Märgendatud fail Tartu_V6nnu_Kiidj2rve_id24500_1866a.json
Märgendatud fail Viru_Rakvere_Rakvere_id5143_1871a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17913_1885a.json
Märgendatud fail Tartu_Kodavere_Pala_id23275_1872a.json
Märgendatud fail L22ne_Reigi_K6rgessaare_id23087_1894a.json
Märgendatud fail Tartu_N6o_Aru_id5374_1890a.json

***** Epoch #23 *****
Loss: 5101.782358
Improvement ratio: 0.296198
Feature L2-norm: 104.573442
Learning rate (eta): 0.049885
Total number of feature updates: 359122
Seconds required for this iteration: 0.249

***** Epoch #24 *****
Loss: 4952.886819
Improvement ratio: 0.258961
Feature L2-norm: 106.345750
Learning rate (eta): 0.049880
Total number of feature updates: 374736
Seconds required for this iteration: 0.248

***** Epoch #25 *****
Loss: 4804.310376
Improvement ratio: 0.307218
Feature L2-norm: 108.021824
Learning rate (eta): 0.049875
Total number of feature updates: 390350
Seconds required for this iteration: 0.242

***** Epoch #26 *****
Loss: 4802.247662
Improvement ratio: 0.233265
Feature L2-norm: 109.680810
Learning rate (eta): 0.049870
Total number of feature updates: 405964
Seconds required for this iteration: 0.248

***** Epoch #27 *****
Loss: 4701.713981
Improvement ratio: 0.219466
Feature L2-norm: 111.414330
Learning rate (eta): 0.049865
Total number of feature updates: 4

Märgendatud fail Tartu_V6nnu_Ahja_id13565_1881a.json
Märgendatud fail Harju_Juuru_Juuru_id17866_1869a.json
Märgendatud fail Harju_Juuru_Kaiu_id931_1912a.json
Märgendatud fail J2rva_Anna_Eivere_id6607_1880a.json
Märgendatud fail L22ne_Kullamaa_Piirsalu_id16723_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id17604_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14708_1882a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id9914_1880a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23461_1873a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21842_1868a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id7633_1870a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id13040_1856a.json
Märgendatud fail Saare_P8ide_Laimjala_id6577_1917a.json
Märgendatud fail J2rva_Tyri_Kirna_id24066_1879a.json
Märgendatud fail L22ne_Ridala_Sinalepa_id25489_1890a.json
Märgendatud fail Tartu_V6nnu_Ahja_id11349_1872a.json
Märgendatud fail Tartu_Kodavere_Ranna_id14234_1857a.json
Märgendatud fail V6ru_P6lva_K2hri_id21791_1851a.json

Märgendatud fail Tartu_Torma_Avinurme_id14255_1903a.json
Märgendatud fail Tartu_Kodavere_Ranna_id15105_1863a.json
Märgendatud fail Harju_Hageri_Kohila_id10294_1868a.json
Märgendatud fail Tartu_Kodavere_Pala_id16727_1855a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id22431_1880a.json
Märgendatud fail Saare_Kihelkonna_Kotlandi_id21549_1869a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18782_1897a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16955_1887a.json
Märgendatud fail J2rva_J2rva-Jaani_Einmanni_id9007_1868a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23568_1895a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1504_1865a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id18104_1877a.json
Märgendatud fail L22ne_Emmaste_Emmaste_id8296_1894a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13380_1881a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id9556_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22940_1889a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id4128_1883a.json
Märgendatud fail J2rv

In [6]:
models = ['model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']
'''
models = ['model_default',\
          'model_local_features_without_morph',\
          'model_morph_with_lemmas',\
          'model_morph_with_lemmas_and_sentences',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features',\
          'model_morph_without_lemmas']
'''
for model_directory in models:
    extract_results_to_txt_file(model_directory, files)

Tulemuste ammutamine on lõpetatud.
